# Plot Average Model State Timeline

##### Supplementary code for Faletti et al. (2026): _Using Ensemble Sensitivity to Diagnose Environmental Modulation of Mesocyclone Intensity in the Warn-on-Forecast System_

### Reproduces Fig. 5

In [1]:
import numpy as np
import xarray as xr
import glob
import os

import matplotlib.pyplot as plt

import haversine as hs

from wofunits import wofunits


# set paths
wofs_path = wofunits.paths['wofs_path']
outplot_path = wofunits.paths['output_paths']

## Process data and compute mean stats

In [2]:
########### PBL SCHEME SPECS ##################

ysu_idx = wofunits.schemeidx['ysu']
myj_idx = wofunits.schemeidx['myj']
mynn_idx = wofunits.schemeidx['mynn']
pbl_list = wofunits.schemeidx['scheme_list']

############ Define files for each case ###############

files_517_2100 = sorted(glob.glob(f'{wofs_path}/wofs_{20190517}_{2100}/*'))
files_517_2200 = sorted(glob.glob(f'{wofs_path}/wofs_{20190517}_{2200}/*'))
files_520 = sorted(glob.glob(f'{wofs_path}/wofs_{20190520}_{2030}/*'))
files_526 = sorted(glob.glob(f'{wofs_path}/wofs_{20190526}_{2000}/*'))
files_528 = sorted(glob.glob(f'{wofs_path}/wofs_{20190528}_{2230}/*'))

files_517_2100 = files_517_2100[11:50][::3]
files_517_2200 = files_517_2200[14:21] + files_517_2200[23:][::3] + files_517_2200[-2:]
files_520 = files_520[13:][::3]
files_526 = files_526[9:13] + files_526[15:-2][::3] + files_526[-2:]
files_528 = files_528[15:][::3]

In [ ]:
ysu_t2_avg = []
myj_t2_avg = []
mynn_t2_avg = []
full_t2_avg = []

ysu_pblh_avg = []
myj_pblh_avg = []
mynn_pblh_avg = []
full_pblh_avg = []

ysu_shr1_avg = []
myj_shr1_avg = []
mynn_shr1_avg = []
full_shr1_avg = []

for i in range(len(files_517_2100)):
    ds = xr.open_mfdataset([files_517_2100[i], files_517_2200[i], files_520[i], files_526[i], files_528[i]], 
                       combine='nested', concat_dim='case')
    t2 = ds['T2'].values
    pblh = ds['TD2'].values
    shr1 = ds['SHEAR_TOT1'].values

    ysu_t2_avg.append(np.nanmean(t2[:,ysu_idx]))
    ysu_pblh_avg.append(np.nanmean(pblh[:,ysu_idx]))
    ysu_shr1_avg.append(np.nanmean(shr1[:,ysu_idx]))

    myj_t2_avg.append(np.nanmean(t2[:,myj_idx]))
    myj_pblh_avg.append(np.nanmean(pblh[:,myj_idx]))
    myj_shr1_avg.append(np.nanmean(shr1[:,myj_idx]))

    mynn_t2_avg.append(np.nanmean(t2[:,mynn_idx]))
    mynn_pblh_avg.append(np.nanmean(pblh[:,mynn_idx]))
    mynn_shr1_avg.append(np.nanmean(shr1[:,mynn_idx]))

    full_t2_avg.append(np.nanmean(t2))
    full_pblh_avg.append(np.nanmean(pblh))
    full_shr1_avg.append(np.nanmean(shr1))

plotvars = [[ysu_t2_avg, myj_t2_avg, mynn_t2_avg, full_t2_avg],
           [ysu_pblh_avg, myj_pblh_avg, mynn_pblh_avg, full_pblh_avg],
           [ysu_shr1_avg, myj_shr1_avg, mynn_shr1_avg, full_shr1_avg]]

var_names = ['2m Temperature', '2m Dewpoint', '0-1 km Shear']

## Plot Fig. 5: Domain-average state through time

In [ ]:
colors = ['tab:orange', 'tab:blue', 'tab:green']

ylabels = ['K', 'K', 'm s$^{-1}$']

sublabels = ['a)', 'b)', 'c)', 'd)']

fig, ax = plt.subplots(1, 3, figsize=(13,3.7))
for j in range(len(plotvars)):
    for i in range(len(plotvars[j])):
        if i < len(plotvars):
            ax[j].plot(np.arange(0, 181, 15), plotvars[j][i], lw=2, c=colors[i])
        if i == len(plotvars):
            ax[j].plot(np.arange(0, 181, 15), plotvars[j][i], c='k', lw=3, ls='--')
        ax[j].set_xticks(np.arange(0, 181, 30))
        ax[j].set_title(var_names[j], size=14, weight='bold')
        ax[j].set_xlabel('Minutes since Initialization', size=12, weight='bold')
        ax[j].set_ylabel(ylabels[j], size=12, weight='bold')
        ax[j].set_xlim(0,180)
        ax[j].text(0.07, 1.05, sublabels[j], transform=ax[j].transAxes, weight='bold',
            fontsize='xx-large', verticalalignment='top', horizontalalignment='right',
            bbox=dict(facecolor='w', edgecolor='k', pad=0.3, lw=1.3, boxstyle='round'), size=15)
        
ax[0].legend(['YSU', 'MYJ', 'MYNN', 'Full'], fontsize=11)

plt.tight_layout()

plt.savefig(f'{outplot_path}/avg_state_through_time.jpg', dpi = 300,
                bbox_inches='tight', pad_inches=0.2, facecolor='w')